In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Thompson,55.7435,-97.8558,73.56,33,40,11.50,CA,scattered clouds
1,1,Concepcion,-36.8270,-73.0498,51.60,62,100,14.97,CL,overcast clouds
2,2,Atuona,-9.8000,-139.0333,76.80,81,29,28.16,PF,light rain
3,3,Bushehr,28.9684,50.8385,91.15,62,0,0.92,IR,clear sky
4,4,Jamestown,42.0970,-79.2353,70.02,74,100,10.36,US,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))


max_temp = float(input("What is the maximum temperature you would like for your trip? "))




What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
 #3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
2,2,Atuona,-9.8000,-139.0333,76.80,81,29,28.16,PF,light rain
13,13,Vaini,-21.2000,-175.2000,78.96,89,20,6.91,TO,few clouds
15,15,Catuday,16.2923,119.8062,79.77,83,74,3.29,PH,broken clouds
19,19,Saint-Gedeon,48.5001,-71.7658,75.11,50,10,2.30,CA,clear sky
22,22,Butaritari,3.0707,172.7902,81.72,72,100,14.70,KI,overcast clouds
25,25,Kloulklubed,7.0419,134.2556,75.70,77,99,5.44,PW,overcast clouds
35,35,Faanui,-16.4833,-151.7500,78.22,74,9,21.18,PF,clear sky
36,36,Pandharpur,17.6667,75.3333,77.85,82,100,5.48,IN,light rain
40,40,Paka,4.6374,103.4368,75.13,94,100,0.00,MY,overcast clouds
44,44,Isangel,-19.5500,169.2667,78.91,83,40,0.00,VU,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                222
City                   222
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Country                222
Current Description    222
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [10]:
# create a new DataFrame that doesn’t have empty rows
clean_df =preferred_cities_df

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,76.80,light rain,-9.8000,-139.0333,
13,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
15,Catuday,PH,79.77,broken clouds,16.2923,119.8062,
19,Saint-Gedeon,CA,75.11,clear sky,48.5001,-71.7658,
22,Butaritari,KI,81.72,overcast clouds,3.0707,172.7902,
25,Kloulklubed,PW,75.70,overcast clouds,7.0419,134.2556,
35,Faanui,PF,78.22,clear sky,-16.4833,-151.7500,
36,Pandharpur,IN,77.85,light rain,17.6667,75.3333,
40,Paka,MY,75.13,overcast clouds,4.6374,103.4368,
44,Isangel,VU,78.91,scattered clouds,-19.5500,169.2667,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
hotel_df.count()

City                   222
Country                222
Max Temp               222
Current Description    222
Lat                    222
Lng                    222
Hotel Name             222
dtype: int64

In [30]:
 #7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,76.80,light rain,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
13,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort
15,Catuday,PH,79.77,broken clouds,16.2923,119.8062,Benjie's Kubo & Cottages
19,Saint-Gedeon,CA,75.11,clear sky,48.5001,-71.7658,Camping Plage St-Jude
22,Butaritari,KI,81.72,overcast clouds,3.0707,172.7902,Isles Sunset Lodge


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file,index_label="Hotel Name")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))